In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset,TensorDataset
from autoencoder import Autoencoder
import torchvision
from model2 import classification_model
import copy
import partition
from pca import PCADigitReducer
from autoencoder import reduce_dimensions
from training import train,test, train_fashion,test_fashion
from federated_learning import distribute_global_model, federated_averaging
from model4 import MultilayerPerceptron
import cluster

# Preprocessing

In [3]:
# Predefined stuff

n_epochs = 5
batch_size_train = 100
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
log_interval = 10
num_clusters = 2

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

In [4]:
fashion_mnist_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.2860,), (0.3204,))  
])

fashion_mnist_train_loader = DataLoader(
    datasets.FashionMNIST('/files/', train=True, download=True, transform=fashion_mnist_transform),
    batch_size=batch_size_train, shuffle=True
)

fashion_mnist_test_loader = DataLoader(
    datasets.FashionMNIST('/files/', train=False, download=True, transform=fashion_mnist_transform),
    batch_size=batch_size_test, shuffle=True
)

In [5]:
train_loader_pca = copy.copy(fashion_mnist_train_loader)
test_loader_pca = copy.copy(fashion_mnist_test_loader)

train_loader_auto = copy.copy(fashion_mnist_train_loader)
test_loader_auto = copy.copy(fashion_mnist_test_loader)

In [6]:
class CustomTensorDataset(TensorDataset):
    def __init__(self, *tensors):
        super().__init__(*tensors)
        self.data = tensors[0]
        self.targets = tensors[1] 

# Pca

In [7]:
train_data = []
train_labels = []
for data, labels in train_loader_pca:
    train_data.append(data.view(data.size(0), -1))  
    train_labels.append(labels)
train_data = torch.cat(train_data, dim=0)  
train_labels = torch.cat(train_labels, dim=0)

train_data_np = train_data.numpy()

pca = PCADigitReducer(100)
train_data_reduced = pca.fit_transform(train_data_np)  

train_data_reconstructed_np = pca.inverse_transform(train_data_reduced) 
train_data_reconstructed = torch.tensor(train_data_reconstructed_np, dtype=torch.float32)

train_data_reconstructed = train_data_reconstructed.view(-1, 1, 28, 28)

train_data_reconstructed = (train_data_reconstructed - 0.2860) / 0.3204

batch_size_train = train_loader_pca.batch_size
train_dataset_pca = CustomTensorDataset(train_data_reconstructed, train_labels)
train_loader_reduced_pca = DataLoader(train_dataset_pca, batch_size=batch_size_train, shuffle=True)

# Autoencoder

In [8]:
# Autoencoder
latent_dim = 100  
autoencoder = Autoencoder(latent_dim=latent_dim)
auto_criterion = nn.MSELoss()
auto_optimizer = optim.Adam(autoencoder.parameters(), lr=1e-3)
auto_num_epochs = 5
for epoch in range(auto_num_epochs): 
    for images, _ in train_loader_auto:
        auto_optimizer.zero_grad()
        reconstructed = autoencoder(images)
        loss = auto_criterion(reconstructed, images)  
        loss.backward()
        auto_optimizer.step()
        
    print(f"Epoch [{epoch+1}/5], Loss: {loss.item()}")

Epoch [1/5], Loss: 0.6599028706550598
Epoch [2/5], Loss: 0.6295812129974365
Epoch [3/5], Loss: 0.6253719329833984
Epoch [4/5], Loss: 0.594176173210144
Epoch [5/5], Loss: 0.623320996761322


In [9]:
autoencoder.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
latent_features, labels = reduce_dimensions(train_loader_auto, autoencoder.encoder, device)
latent_features = latent_features.detach()

reconstructed_images = autoencoder.decoder(latent_features.to(device))  
reconstructed_images = reconstructed_images.view(-1, 1, 28, 28)  # Reshape to [batch_size, channels, height, width]

reconstructed_dataset = CustomTensorDataset(reconstructed_images.cpu(), labels)  
reduced_train_loader_auto = DataLoader(reconstructed_dataset, batch_size=batch_size_train, shuffle=True)

# Partition

## Classic Dataloaders

In [10]:
# classic
trainingset = fashion_mnist_train_loader.dataset
partitioned_data_classic = partition.balanced_dirichlet_partition(trainingset, partitions_number=4, alpha=0.5)

In [31]:
import cluster
cluster = cluster.Cluster(num_clusters=4)

targets = trainingset.targets
num_classes = len(set(targets)) 
clustered_data = cluster.apply_clustering(partitioned_data_classic, targets, num_classes)

partitioned_data_classic_clustered = clustered_data

In [12]:
# Normal loader classic
classic_client_loaders = [
    DataLoader(Subset(trainingset, indices), batch_size=batch_size_train, shuffle=True)
    for indices in partitioned_data_classic.values()
]

In [32]:
# Clustered loader classic
classic_client_loaders_clustered = [
    DataLoader(Subset(trainingset, indices), batch_size=batch_size_train, shuffle=True)
    for indices in partitioned_data_classic_clustered.values()
]

In [33]:
print(len(classic_client_loaders_clustered))

4


## PCA Dataloaders

In [14]:
# pca 
trainingset_pca = train_loader_reduced_pca.dataset
partitioned_data_pca = partition.balanced_dirichlet_partition(trainingset_pca, partitions_number=4, alpha=0.5)

In [34]:
import cluster
cluster = cluster.Cluster(num_clusters=4)

targets = trainingset_pca.targets
num_classes = len(set(targets)) 
clustered_data = cluster.apply_clustering(partitioned_data_pca, targets, num_classes)

partitioned_data_pca_clustered = clustered_data

In [16]:
# Normal loader pca
pca_client_loaders = [
    DataLoader(Subset(trainingset_pca, indices), batch_size=batch_size_train, shuffle=True)
    for indices in partitioned_data_pca.values()
]

In [35]:
# Clustered loader pca
pca_client_loaders_clustered = [
    DataLoader(Subset(trainingset_pca, indices), batch_size=batch_size_train, shuffle=True)
    for indices in partitioned_data_pca_clustered.values()
]

In [36]:
print(len(pca_client_loaders_clustered))

4


## Autoencoder Dataloaders

In [18]:
# Autoencoder
trainingset_auto = reduced_train_loader_auto.dataset
print(trainingset_auto.targets)
print(trainingset_auto.data.shape)
partitioned_data_auto = partition.balanced_dirichlet_partition(trainingset_auto, partitions_number=4, alpha=0.5)

tensor([6, 3, 2,  ..., 0, 6, 8])
torch.Size([60000, 1, 28, 28])


In [37]:
import cluster
cluster = cluster.Cluster(num_clusters=4)

targets = trainingset_auto.targets
num_classes = len(set(targets)) 
clustered_data = cluster.apply_clustering(partitioned_data_auto, targets, num_classes)

partitioned_data_auto_clustered = clustered_data

In [20]:
# Normal loader autoencoder
auto_client_loaders = [
    DataLoader(Subset(trainingset_auto, indices), batch_size=batch_size_train, shuffle=True)
    for indices in partitioned_data_auto.values()
]

In [38]:
# Clustered loader autoencoder
auto_client_loaders_clustered = [
    DataLoader(Subset(trainingset_auto, indices), batch_size=batch_size_train, shuffle=True)
    for indices in partitioned_data_auto_clustered.values()
]

In [39]:
print(len(auto_client_loaders_clustered))

4


# Models

In [22]:
# Weak model
trial_model = classification_model()
trial_model_pca = classification_model()
trial_model_auto = classification_model()

global_model_pca = classification_model()
global_model_auto = classification_model()
global_model_classic = classification_model()

num_clients = 4
# classic models
local_models_classic = [copy.deepcopy(global_model_pca) for _ in range(num_clients)]
# pca models 
local_models_pca = [copy.deepcopy(global_model_pca) for _ in range(num_clients)]
# autoencodere models
local_model_autoencoder = [copy.deepcopy(global_model_pca) for _ in range(num_clients)]

In [23]:
# Stronger model

trial_model_strong = MultilayerPerceptron()
trial_model_pca_strong = MultilayerPerceptron()
trial_model_auto_strong = MultilayerPerceptron()

global_model_pca_strong = MultilayerPerceptron()
global_model_auto_strong = MultilayerPerceptron()
global_model_classic_strong = MultilayerPerceptron()

num_clients = 4
# classic models
local_models_classic_strong = [copy.deepcopy(global_model_classic_strong) for _ in range(num_clients)]
# pca models 
local_models_pca_strong = [copy.deepcopy(global_model_pca_strong) for _ in range(num_clients)]
# autoencodere models
local_model_autoencoder_strong = [copy.deepcopy(global_model_auto_strong) for _ in range(num_clients)]

# Training 

In [23]:
# test for errors

# Classic weak

optimizer = optim.SGD(trial_model.parameters(), lr=learning_rate,
                      momentum=momentum)

train_losses = []
train_counter = []

for epoch in range(1, n_epochs + 1):  
    train(epoch, trial_model, fashion_mnist_train_loader, optimizer, log_interval, train_losses, train_counter)

c:\Users\micha\Downloads\Federated-Dimensionality-Reduction\model2.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.304462
Train Epoch: 1 [1000/60000 (2%)]	Loss: 2.307527
Train Epoch: 1 [2000/60000 (3%)]	Loss: 2.274717
Train Epoch: 1 [3000/60000 (5%)]	Loss: 2.304101
Train Epoch: 1 [4000/60000 (7%)]	Loss: 2.264609
Train Epoch: 1 [5000/60000 (8%)]	Loss: 2.276606
Train Epoch: 1 [6000/60000 (10%)]	Loss: 2.242934
Train Epoch: 1 [7000/60000 (12%)]	Loss: 2.221632
Train Epoch: 1 [8000/60000 (13%)]	Loss: 2.156260
Train Epoch: 1 [9000/60000 (15%)]	Loss: 2.162500
Train Epoch: 1 [10000/60000 (17%)]	Loss: 2.046984
Train Epoch: 1 [11000/60000 (18%)]	Loss: 2.064728
Train Epoch: 1 [12000/60000 (20%)]	Loss: 2.047116
Train Epoch: 1 [13000/60000 (22%)]	Loss: 1.810572
Train Epoch: 1 [14000/60000 (23%)]	Loss: 1.822670
Train Epoch: 1 [15000/60000 (25%)]	Loss: 1.702481
Train Epoch: 1 [16000/60000 (27%)]	Loss: 1.601983
Train Epoch: 1 [17000/60000 (28%)]	Loss: 1.623126
Train Epoch: 1 [18000/60000 (30%)]	Loss: 1.578819
Train Epoch: 1 [19000/60000 (32%)]	Loss: 1.581380
Train Epoch: 1 [200

In [24]:
test_losses_classic_weak = []
test(trial_model,fashion_mnist_test_loader,test_losses_classic_weak)

c:\Users\micha\anaconda3\Lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: 0.5092, Accuracy: 8112/10000 (81%)



In [39]:
# Pca weak

optimizer = optim.SGD(trial_model_pca.parameters(), lr=learning_rate,
                      momentum=momentum)

train_losses = []
train_counter = []

for epoch in range(1, n_epochs + 1):  
    train(epoch, trial_model_pca, train_loader_reduced_pca, optimizer, log_interval, train_losses, train_counter)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.402107
Train Epoch: 1 [1000/60000 (2%)]	Loss: 2.254441
Train Epoch: 1 [2000/60000 (3%)]	Loss: 2.114528
Train Epoch: 1 [3000/60000 (5%)]	Loss: 2.049475
Train Epoch: 1 [4000/60000 (7%)]	Loss: 1.850434
Train Epoch: 1 [5000/60000 (8%)]	Loss: 1.800493
Train Epoch: 1 [6000/60000 (10%)]	Loss: 1.512475
Train Epoch: 1 [7000/60000 (12%)]	Loss: 1.562135
Train Epoch: 1 [8000/60000 (13%)]	Loss: 1.563771
Train Epoch: 1 [9000/60000 (15%)]	Loss: 1.369700
Train Epoch: 1 [10000/60000 (17%)]	Loss: 1.283631
Train Epoch: 1 [11000/60000 (18%)]	Loss: 1.267053
Train Epoch: 1 [12000/60000 (20%)]	Loss: 1.328841
Train Epoch: 1 [13000/60000 (22%)]	Loss: 1.113709
Train Epoch: 1 [14000/60000 (23%)]	Loss: 1.098118
Train Epoch: 1 [15000/60000 (25%)]	Loss: 1.201959
Train Epoch: 1 [16000/60000 (27%)]	Loss: 1.295908
Train Epoch: 1 [17000/60000 (28%)]	Loss: 1.125021
Train Epoch: 1 [18000/60000 (30%)]	Loss: 1.202553
Train Epoch: 1 [19000/60000 (32%)]	Loss: 1.182726
Train Epoch: 1 [200

In [40]:
test_losses_classic_pca = []
test(trial_model_pca,train_loader_reduced_pca,test_losses_classic_pca)


Test set: Avg. loss: 0.4446, Accuracy: 50317/60000 (84%)



In [41]:
# Auto weak

optimizer = optim.SGD(trial_model_auto.parameters(), lr=learning_rate,
                      momentum=momentum)

train_losses = []
train_counter = []

for epoch in range(1, n_epochs + 1):  
    train(epoch, trial_model_auto, reduced_train_loader_auto, optimizer, log_interval, train_losses, train_counter)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.304784
Train Epoch: 1 [1000/60000 (2%)]	Loss: 2.295412
Train Epoch: 1 [2000/60000 (3%)]	Loss: 2.302005
Train Epoch: 1 [3000/60000 (5%)]	Loss: 2.306511
Train Epoch: 1 [4000/60000 (7%)]	Loss: 2.299275
Train Epoch: 1 [5000/60000 (8%)]	Loss: 2.299407
Train Epoch: 1 [6000/60000 (10%)]	Loss: 2.313645
Train Epoch: 1 [7000/60000 (12%)]	Loss: 2.292254
Train Epoch: 1 [8000/60000 (13%)]	Loss: 2.289868
Train Epoch: 1 [9000/60000 (15%)]	Loss: 2.282523
Train Epoch: 1 [10000/60000 (17%)]	Loss: 2.269839
Train Epoch: 1 [11000/60000 (18%)]	Loss: 2.279874
Train Epoch: 1 [12000/60000 (20%)]	Loss: 2.273515
Train Epoch: 1 [13000/60000 (22%)]	Loss: 2.252221
Train Epoch: 1 [14000/60000 (23%)]	Loss: 2.232630
Train Epoch: 1 [15000/60000 (25%)]	Loss: 2.222997
Train Epoch: 1 [16000/60000 (27%)]	Loss: 2.188857
Train Epoch: 1 [17000/60000 (28%)]	Loss: 2.190028
Train Epoch: 1 [18000/60000 (30%)]	Loss: 2.152280
Train Epoch: 1 [19000/60000 (32%)]	Loss: 2.105967
Train Epoch: 1 [200

In [42]:
test_losses_classic_auto = []
test(trial_model_auto,reduced_train_loader_auto,test_losses_classic_auto)


Test set: Avg. loss: 0.6800, Accuracy: 44727/60000 (75%)



In [43]:
# Classic strong

optimizer = optim.SGD(trial_model_strong.parameters(), lr=learning_rate,
                      momentum=momentum)

train_losses = []
train_counter = []

for epoch in range(1, n_epochs + 1):  
    train_fashion(epoch, trial_model_strong, fashion_mnist_train_loader, optimizer, log_interval, train_losses, train_counter)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.300699
Train Epoch: 1 [1000/60000 (2%)]	Loss: 2.204830
Train Epoch: 1 [2000/60000 (3%)]	Loss: 2.086071
Train Epoch: 1 [3000/60000 (5%)]	Loss: 1.952864
Train Epoch: 1 [4000/60000 (7%)]	Loss: 1.704459
Train Epoch: 1 [5000/60000 (8%)]	Loss: 1.530586
Train Epoch: 1 [6000/60000 (10%)]	Loss: 1.370961
Train Epoch: 1 [7000/60000 (12%)]	Loss: 1.155327
Train Epoch: 1 [8000/60000 (13%)]	Loss: 1.056469
Train Epoch: 1 [9000/60000 (15%)]	Loss: 1.046260
Train Epoch: 1 [10000/60000 (17%)]	Loss: 1.043569
Train Epoch: 1 [11000/60000 (18%)]	Loss: 0.794653
Train Epoch: 1 [12000/60000 (20%)]	Loss: 0.821681
Train Epoch: 1 [13000/60000 (22%)]	Loss: 0.925488
Train Epoch: 1 [14000/60000 (23%)]	Loss: 0.843757
Train Epoch: 1 [15000/60000 (25%)]	Loss: 0.887245
Train Epoch: 1 [16000/60000 (27%)]	Loss: 0.790194
Train Epoch: 1 [17000/60000 (28%)]	Loss: 0.753052
Train Epoch: 1 [18000/60000 (30%)]	Loss: 0.737370
Train Epoch: 1 [19000/60000 (32%)]	Loss: 0.669351
Train Epoch: 1 [200

In [44]:
test_losses_classic_strong = []
test_fashion(trial_model_strong,fashion_mnist_train_loader,test_losses_classic_strong)


Test set: Avg. loss: 0.3646, Accuracy: 52149/60000 (87%)



In [45]:
# Pca strong

optimizer = optim.SGD(trial_model_pca_strong.parameters(), lr=learning_rate,
                      momentum=momentum)

train_losses = []
train_counter = []

for epoch in range(1, n_epochs + 1):  
    train_fashion(epoch, trial_model_pca_strong, train_loader_reduced_pca, optimizer, log_interval, train_losses, train_counter)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.401436
Train Epoch: 1 [1000/60000 (2%)]	Loss: 1.713010
Train Epoch: 1 [2000/60000 (3%)]	Loss: 1.331232
Train Epoch: 1 [3000/60000 (5%)]	Loss: 0.870442
Train Epoch: 1 [4000/60000 (7%)]	Loss: 0.815639
Train Epoch: 1 [5000/60000 (8%)]	Loss: 0.653508
Train Epoch: 1 [6000/60000 (10%)]	Loss: 0.835060
Train Epoch: 1 [7000/60000 (12%)]	Loss: 0.670965
Train Epoch: 1 [8000/60000 (13%)]	Loss: 0.690118
Train Epoch: 1 [9000/60000 (15%)]	Loss: 0.571057
Train Epoch: 1 [10000/60000 (17%)]	Loss: 0.571716
Train Epoch: 1 [11000/60000 (18%)]	Loss: 0.472981
Train Epoch: 1 [12000/60000 (20%)]	Loss: 0.487728
Train Epoch: 1 [13000/60000 (22%)]	Loss: 0.646880
Train Epoch: 1 [14000/60000 (23%)]	Loss: 0.694721
Train Epoch: 1 [15000/60000 (25%)]	Loss: 0.470131
Train Epoch: 1 [16000/60000 (27%)]	Loss: 0.774336
Train Epoch: 1 [17000/60000 (28%)]	Loss: 0.556044
Train Epoch: 1 [18000/60000 (30%)]	Loss: 0.674577
Train Epoch: 1 [19000/60000 (32%)]	Loss: 0.541306
Train Epoch: 1 [200

In [46]:
test_losses_pca_strong = []
test_fashion(trial_model_pca_strong,train_loader_reduced_pca,test_losses_pca_strong)


Test set: Avg. loss: 0.3288, Accuracy: 52742/60000 (88%)



In [47]:
# Auto strong

optimizer = optim.SGD(trial_model_auto_strong.parameters(), lr=learning_rate,
                      momentum=momentum)

train_losses = []
train_counter = []

for epoch in range(1, n_epochs + 1):  
    train_fashion(epoch, trial_model_auto_strong, reduced_train_loader_auto, optimizer, log_interval, train_losses, train_counter)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.303825
Train Epoch: 1 [1000/60000 (2%)]	Loss: 2.273778
Train Epoch: 1 [2000/60000 (3%)]	Loss: 2.248430
Train Epoch: 1 [3000/60000 (5%)]	Loss: 2.210776
Train Epoch: 1 [4000/60000 (7%)]	Loss: 2.168177
Train Epoch: 1 [5000/60000 (8%)]	Loss: 2.138035
Train Epoch: 1 [6000/60000 (10%)]	Loss: 2.066162
Train Epoch: 1 [7000/60000 (12%)]	Loss: 2.009484
Train Epoch: 1 [8000/60000 (13%)]	Loss: 1.986999
Train Epoch: 1 [9000/60000 (15%)]	Loss: 1.827304
Train Epoch: 1 [10000/60000 (17%)]	Loss: 1.805411
Train Epoch: 1 [11000/60000 (18%)]	Loss: 1.776352
Train Epoch: 1 [12000/60000 (20%)]	Loss: 1.657033
Train Epoch: 1 [13000/60000 (22%)]	Loss: 1.575783
Train Epoch: 1 [14000/60000 (23%)]	Loss: 1.541192
Train Epoch: 1 [15000/60000 (25%)]	Loss: 1.489067
Train Epoch: 1 [16000/60000 (27%)]	Loss: 1.438832
Train Epoch: 1 [17000/60000 (28%)]	Loss: 1.330647
Train Epoch: 1 [18000/60000 (30%)]	Loss: 1.375636
Train Epoch: 1 [19000/60000 (32%)]	Loss: 1.331015
Train Epoch: 1 [200

In [48]:
test_losses_auto_strong = []
test_fashion(trial_model_auto_strong,reduced_train_loader_auto,test_losses_auto_strong)


Test set: Avg. loss: 0.5501, Accuracy: 48284/60000 (80%)



## Weak Model

In [36]:
# Classic

rounds_classic = 4

for round_idx in range(rounds_classic):
    
    print(f"Round {round_idx + 1}/{rounds_classic}")

    local_weights_classic = []
    for client_idx, client_model in enumerate(local_models_classic):
        print(f"Training client {client_idx + 1}")
        
        optimizer = optim.SGD(client_model.parameters(), lr=learning_rate,
                      momentum=momentum)

        train_losses = []
        train_counter = []

        for epoch in range(1, n_epochs + 1):  
            train(epoch, client_model, classic_client_loaders[client_idx], optimizer, log_interval, train_losses, train_counter)
        
        client_weights = [param.data.numpy() for param in client_model.parameters()]
        local_weights_classic.append(client_weights)
        
    print(f"after training{local_models_classic}")
    global_weights_classic = federated_averaging(local_weights_classic)
    print(f"after fedaveraging{local_models_classic}")

    distribute_global_model(global_weights_classic,local_models_classic,single=False)

    distribute_global_model(global_weights_classic,global_model_classic,single=True)
    test_losses = []
    test(global_model_classic,fashion_mnist_test_loader,test_losses)

Round 1/4
Training client 1
Train Epoch: 1 [0/23974 (0%)]	Loss: 2.311635
Train Epoch: 1 [1000/23974 (4%)]	Loss: 2.267546
Train Epoch: 1 [2000/23974 (8%)]	Loss: 2.172202
Train Epoch: 1 [3000/23974 (12%)]	Loss: 2.187181
Train Epoch: 1 [4000/23974 (17%)]	Loss: 2.052589
Train Epoch: 1 [5000/23974 (21%)]	Loss: 2.086102
Train Epoch: 1 [6000/23974 (25%)]	Loss: 1.958034
Train Epoch: 1 [7000/23974 (29%)]	Loss: 1.853657
Train Epoch: 1 [8000/23974 (33%)]	Loss: 1.870320
Train Epoch: 1 [9000/23974 (38%)]	Loss: 1.753847
Train Epoch: 1 [10000/23974 (42%)]	Loss: 1.721817
Train Epoch: 1 [11000/23974 (46%)]	Loss: 1.611107
Train Epoch: 1 [12000/23974 (50%)]	Loss: 1.550352
Train Epoch: 1 [13000/23974 (54%)]	Loss: 1.510496
Train Epoch: 1 [14000/23974 (58%)]	Loss: 1.440019
Train Epoch: 1 [15000/23974 (62%)]	Loss: 1.374621
Train Epoch: 1 [16000/23974 (67%)]	Loss: 1.339348
Train Epoch: 1 [17000/23974 (71%)]	Loss: 1.371690
Train Epoch: 1 [18000/23974 (75%)]	Loss: 1.260237
Train Epoch: 1 [19000/23974 (79%)]	Los

In [35]:
# PCA

rounds_pca = 4

for round_idx in range(rounds_pca):
    
    print(f"Round {round_idx + 1}/{rounds_pca}")

    local_weights_pca = []
    for client_idx, client_model in enumerate(local_models_pca):
        print(f"Training client {client_idx + 1}")
        
        optimizer = optim.SGD(client_model.parameters(), lr=learning_rate,
                      momentum=momentum)

        train_losses = []
        train_counter = []


        for epoch in range(1, n_epochs + 1):  
            train(epoch, client_model, pca_client_loaders[client_idx], optimizer, log_interval, train_losses, train_counter)
        
        client_weights = [param.data.numpy() for param in client_model.parameters()]
        local_weights_pca.append(client_weights)
        
    print(f"after training{local_models_pca}")
    global_weights_pca = federated_averaging(local_weights_pca)
    print(f"after fedaveraging{local_models_pca}")

    distribute_global_model(global_weights_pca,local_models_pca,single=False)

    distribute_global_model(global_weights_pca,global_model_pca,single=True)
    test_losses = []
    test(global_model_pca,test_loader_pca,test_losses)

Round 1/4
Training client 1
Train Epoch: 1 [0/23974 (0%)]	Loss: 2.424798
Train Epoch: 1 [1000/23974 (4%)]	Loss: 2.195513
Train Epoch: 1 [2000/23974 (8%)]	Loss: 2.117115
Train Epoch: 1 [3000/23974 (12%)]	Loss: 1.872272
Train Epoch: 1 [4000/23974 (17%)]	Loss: 1.844926
Train Epoch: 1 [5000/23974 (21%)]	Loss: 1.587915
Train Epoch: 1 [6000/23974 (25%)]	Loss: 1.416638
Train Epoch: 1 [7000/23974 (29%)]	Loss: 1.322630
Train Epoch: 1 [8000/23974 (33%)]	Loss: 1.318197
Train Epoch: 1 [9000/23974 (38%)]	Loss: 1.275360
Train Epoch: 1 [10000/23974 (42%)]	Loss: 1.303297
Train Epoch: 1 [11000/23974 (46%)]	Loss: 0.903797
Train Epoch: 1 [12000/23974 (50%)]	Loss: 0.926429
Train Epoch: 1 [13000/23974 (54%)]	Loss: 0.983554
Train Epoch: 1 [14000/23974 (58%)]	Loss: 1.091592
Train Epoch: 1 [15000/23974 (62%)]	Loss: 1.074514
Train Epoch: 1 [16000/23974 (67%)]	Loss: 1.051557
Train Epoch: 1 [17000/23974 (71%)]	Loss: 1.014256
Train Epoch: 1 [18000/23974 (75%)]	Loss: 0.780286
Train Epoch: 1 [19000/23974 (79%)]	Los

In [37]:
# Autoencoder

rounds_auto = 4

for round_idx in range(rounds_auto):
    print(f"Round {round_idx + 1}/{rounds_auto}")

    local_weights_auto = []
    for client_idx, client_model in enumerate(local_model_autoencoder):
        print(f"Training client {client_idx + 1}")
        
        optimizer = optim.SGD(client_model.parameters(), lr=learning_rate,
                      momentum=momentum)
        
        train_losses = []
        train_counter = []

        for epoch in range(1, n_epochs + 1):  
            train(epoch, client_model, auto_client_loaders[client_idx], optimizer, log_interval, train_losses, train_counter)
        
        client_weights = [param.data.numpy() for param in client_model.parameters()]
        local_weights_auto.append(client_weights)
        
    global_weights_auto = federated_averaging(local_weights_auto)

    distribute_global_model(global_weights_auto,local_model_autoencoder,single=False)

    distribute_global_model(global_weights_auto, global_model_auto,single=True)
    test_losses = []
    test(global_model_auto,test_loader_auto,test_losses)

Round 1/4
Training client 1
Train Epoch: 1 [0/23974 (0%)]	Loss: 2.342116
Train Epoch: 1 [1000/23974 (4%)]	Loss: 2.268985
Train Epoch: 1 [2000/23974 (8%)]	Loss: 2.251395
Train Epoch: 1 [3000/23974 (12%)]	Loss: 2.208130
Train Epoch: 1 [4000/23974 (17%)]	Loss: 2.227149
Train Epoch: 1 [5000/23974 (21%)]	Loss: 2.165110
Train Epoch: 1 [6000/23974 (25%)]	Loss: 2.218787
Train Epoch: 1 [7000/23974 (29%)]	Loss: 2.221504
Train Epoch: 1 [8000/23974 (33%)]	Loss: 2.155954
Train Epoch: 1 [9000/23974 (38%)]	Loss: 2.117171
Train Epoch: 1 [10000/23974 (42%)]	Loss: 2.080258
Train Epoch: 1 [11000/23974 (46%)]	Loss: 2.123074
Train Epoch: 1 [12000/23974 (50%)]	Loss: 1.972604
Train Epoch: 1 [13000/23974 (54%)]	Loss: 1.985338
Train Epoch: 1 [14000/23974 (58%)]	Loss: 1.986528
Train Epoch: 1 [15000/23974 (62%)]	Loss: 1.869018
Train Epoch: 1 [16000/23974 (67%)]	Loss: 1.844700
Train Epoch: 1 [17000/23974 (71%)]	Loss: 1.914017
Train Epoch: 1 [18000/23974 (75%)]	Loss: 1.827646
Train Epoch: 1 [19000/23974 (79%)]	Los

## Strong model

In [24]:
# Classic

rounds_classic = 4

for round_idx in range(rounds_classic):
    
    print(f"Round {round_idx + 1}/{rounds_classic}")

    local_weights_classic = []
    for client_idx, client_model in enumerate(local_models_classic_strong):
        print(f"Training client {client_idx + 1}")
        
        optimizer = optim.SGD(client_model.parameters(), lr=learning_rate,
                      momentum=momentum)

        train_losses = []
        train_counter = []

        for epoch in range(1, n_epochs + 1):  
            train_fashion(epoch, client_model, classic_client_loaders[client_idx], optimizer, log_interval, train_losses, train_counter)
        
        client_weights = [param.data.numpy() for param in client_model.parameters()]
        local_weights_classic.append(client_weights)
        

    global_weights_classic = federated_averaging(local_weights_classic)


    distribute_global_model(global_weights_classic,local_models_classic_strong,single=False)

    distribute_global_model(global_weights_classic,global_model_classic_strong,single=True)
    test_losses = []
    test_fashion(global_model_classic_strong,fashion_mnist_test_loader,test_losses)

Round 1/4
Training client 1
Train Epoch: 1 [0/23974 (0%)]	Loss: 2.282033
Train Epoch: 1 [1000/23974 (4%)]	Loss: 2.087952
Train Epoch: 1 [2000/23974 (8%)]	Loss: 1.822377
Train Epoch: 1 [3000/23974 (12%)]	Loss: 1.587584
Train Epoch: 1 [4000/23974 (17%)]	Loss: 1.329372
Train Epoch: 1 [5000/23974 (21%)]	Loss: 1.220368
Train Epoch: 1 [6000/23974 (25%)]	Loss: 1.174772
Train Epoch: 1 [7000/23974 (29%)]	Loss: 1.170583
Train Epoch: 1 [8000/23974 (33%)]	Loss: 0.940630
Train Epoch: 1 [9000/23974 (38%)]	Loss: 0.855867
Train Epoch: 1 [10000/23974 (42%)]	Loss: 0.776116
Train Epoch: 1 [11000/23974 (46%)]	Loss: 0.599810
Train Epoch: 1 [12000/23974 (50%)]	Loss: 0.540728
Train Epoch: 1 [13000/23974 (54%)]	Loss: 0.856453
Train Epoch: 1 [14000/23974 (58%)]	Loss: 0.712915
Train Epoch: 1 [15000/23974 (62%)]	Loss: 0.579887
Train Epoch: 1 [16000/23974 (67%)]	Loss: 0.599526
Train Epoch: 1 [17000/23974 (71%)]	Loss: 0.600913
Train Epoch: 1 [18000/23974 (75%)]	Loss: 0.672506
Train Epoch: 1 [19000/23974 (79%)]	Los

c:\Users\micha\anaconda3\Lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: 0.6871, Accuracy: 7442/10000 (74%)

Round 2/4
Training client 1
Train Epoch: 1 [0/23974 (0%)]	Loss: 0.555262
Train Epoch: 1 [1000/23974 (4%)]	Loss: 0.484562
Train Epoch: 1 [2000/23974 (8%)]	Loss: 0.403853
Train Epoch: 1 [3000/23974 (12%)]	Loss: 0.469763
Train Epoch: 1 [4000/23974 (17%)]	Loss: 0.324605
Train Epoch: 1 [5000/23974 (21%)]	Loss: 0.440086
Train Epoch: 1 [6000/23974 (25%)]	Loss: 0.502901
Train Epoch: 1 [7000/23974 (29%)]	Loss: 0.598377
Train Epoch: 1 [8000/23974 (33%)]	Loss: 0.494640
Train Epoch: 1 [9000/23974 (38%)]	Loss: 0.376995
Train Epoch: 1 [10000/23974 (42%)]	Loss: 0.427841
Train Epoch: 1 [11000/23974 (46%)]	Loss: 0.404094
Train Epoch: 1 [12000/23974 (50%)]	Loss: 0.406582
Train Epoch: 1 [13000/23974 (54%)]	Loss: 0.536891
Train Epoch: 1 [14000/23974 (58%)]	Loss: 0.385792
Train Epoch: 1 [15000/23974 (62%)]	Loss: 0.387168
Train Epoch: 1 [16000/23974 (67%)]	Loss: 0.486484
Train Epoch: 1 [17000/23974 (71%)]	Loss: 0.392895
Train Epoch: 1 [18000/23974 (7

In [25]:
# PCA

rounds_pca = 4

for round_idx in range(rounds_pca):
    
    print(f"Round {round_idx + 1}/{rounds_pca}")

    local_weights_pca = []
    for client_idx, client_model in enumerate(local_models_pca_strong):
        print(f"Training client {client_idx + 1}")
        
        optimizer = optim.SGD(client_model.parameters(), lr=learning_rate,
                      momentum=momentum)

        train_losses = []
        train_counter = []


        for epoch in range(1, n_epochs + 1):  
            train_fashion(epoch, client_model, pca_client_loaders[client_idx], optimizer, log_interval, train_losses, train_counter)
        
        client_weights = [param.data.numpy() for param in client_model.parameters()]
        local_weights_pca.append(client_weights)
        

    global_weights_pca = federated_averaging(local_weights_pca)

    distribute_global_model(global_weights_pca,local_models_pca_strong,single=False)

    distribute_global_model(global_weights_pca,global_model_pca_strong,single=True)
    test_losses = []
    test_fashion(global_model_pca_strong,test_loader_pca,test_losses)

Round 1/4
Training client 1
Train Epoch: 1 [0/23974 (0%)]	Loss: 2.302472
Train Epoch: 1 [1000/23974 (4%)]	Loss: 1.339018
Train Epoch: 1 [2000/23974 (8%)]	Loss: 0.878476
Train Epoch: 1 [3000/23974 (12%)]	Loss: 0.620316
Train Epoch: 1 [4000/23974 (17%)]	Loss: 0.594155
Train Epoch: 1 [5000/23974 (21%)]	Loss: 0.547061
Train Epoch: 1 [6000/23974 (25%)]	Loss: 0.588864
Train Epoch: 1 [7000/23974 (29%)]	Loss: 0.651196
Train Epoch: 1 [8000/23974 (33%)]	Loss: 0.659951
Train Epoch: 1 [9000/23974 (38%)]	Loss: 0.524324
Train Epoch: 1 [10000/23974 (42%)]	Loss: 0.639266
Train Epoch: 1 [11000/23974 (46%)]	Loss: 0.561694
Train Epoch: 1 [12000/23974 (50%)]	Loss: 0.434012
Train Epoch: 1 [13000/23974 (54%)]	Loss: 0.460489
Train Epoch: 1 [14000/23974 (58%)]	Loss: 0.669259
Train Epoch: 1 [15000/23974 (62%)]	Loss: 0.478719
Train Epoch: 1 [16000/23974 (67%)]	Loss: 0.446983
Train Epoch: 1 [17000/23974 (71%)]	Loss: 0.338960
Train Epoch: 1 [18000/23974 (75%)]	Loss: 0.517846
Train Epoch: 1 [19000/23974 (79%)]	Los

In [26]:
# Autoencoder

rounds_auto = 4

for round_idx in range(rounds_auto):
    print(f"Round {round_idx + 1}/{rounds_auto}")

    local_weights_auto = []
    for client_idx, client_model in enumerate(local_model_autoencoder_strong):
        print(f"Training client {client_idx + 1}")
        
        optimizer = optim.SGD(client_model.parameters(), lr=learning_rate,
                      momentum=momentum)
        
        train_losses = []
        train_counter = []

        for epoch in range(1, n_epochs + 1):  
            train_fashion(epoch, client_model, auto_client_loaders[client_idx], optimizer, log_interval, train_losses, train_counter)
        
        client_weights = [param.data.numpy() for param in client_model.parameters()]
        local_weights_auto.append(client_weights)
        
    global_weights_auto = federated_averaging(local_weights_auto)

    distribute_global_model(global_weights_auto,local_model_autoencoder_strong,single=False)

    distribute_global_model(global_weights_auto, global_model_auto_strong,single=True)
    test_losses = []
    test_fashion(global_model_auto_strong,test_loader_auto,test_losses)

Round 1/4
Training client 1
Train Epoch: 1 [0/23974 (0%)]	Loss: 2.304508
Train Epoch: 1 [1000/23974 (4%)]	Loss: 2.256542
Train Epoch: 1 [2000/23974 (8%)]	Loss: 2.208084
Train Epoch: 1 [3000/23974 (12%)]	Loss: 2.101736
Train Epoch: 1 [4000/23974 (17%)]	Loss: 2.018898
Train Epoch: 1 [5000/23974 (21%)]	Loss: 1.893561
Train Epoch: 1 [6000/23974 (25%)]	Loss: 1.903691
Train Epoch: 1 [7000/23974 (29%)]	Loss: 1.689991
Train Epoch: 1 [8000/23974 (33%)]	Loss: 1.692309
Train Epoch: 1 [9000/23974 (38%)]	Loss: 1.554281
Train Epoch: 1 [10000/23974 (42%)]	Loss: 1.469193
Train Epoch: 1 [11000/23974 (46%)]	Loss: 1.430659
Train Epoch: 1 [12000/23974 (50%)]	Loss: 1.373816
Train Epoch: 1 [13000/23974 (54%)]	Loss: 1.353841
Train Epoch: 1 [14000/23974 (58%)]	Loss: 1.274188
Train Epoch: 1 [15000/23974 (62%)]	Loss: 1.263534
Train Epoch: 1 [16000/23974 (67%)]	Loss: 1.016249
Train Epoch: 1 [17000/23974 (71%)]	Loss: 1.315045
Train Epoch: 1 [18000/23974 (75%)]	Loss: 1.014238
Train Epoch: 1 [19000/23974 (79%)]	Los

# Cluster

In [40]:
# Classic

rounds_classic = 4

for round_idx in range(rounds_classic):
    
    print(f"Round {round_idx + 1}/{rounds_classic}")

    local_weights_classic = []
    for client_idx, client_model in enumerate(local_models_classic):
        print(f"Training client {client_idx + 1}")
        
        optimizer = optim.SGD(client_model.parameters(), lr=learning_rate,
                      momentum=momentum)

        train_losses = []
        train_counter = []

        for epoch in range(1, n_epochs + 1):  
            train(epoch, client_model, classic_client_loaders_clustered[client_idx], optimizer, log_interval, train_losses, train_counter)
        
        client_weights = [param.data.numpy() for param in client_model.parameters()]
        local_weights_classic.append(client_weights)
        
    print(f"after training{local_models_classic}")
    global_weights_classic = federated_averaging(local_weights_classic)
    print(f"after fedaveraging{local_models_classic}")

    distribute_global_model(global_weights_classic,local_models_classic,single=False)

    distribute_global_model(global_weights_classic,global_model_classic,single=True)
    test_losses = []
    test(global_model_classic,fashion_mnist_test_loader,test_losses)

Round 1/4
Training client 1
Train Epoch: 1 [0/23974 (0%)]	Loss: 0.456145


c:\Users\micha\Downloads\Federated-Dimensionality-Reduction\model2.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Train Epoch: 1 [1000/23974 (4%)]	Loss: 0.603035
Train Epoch: 1 [2000/23974 (8%)]	Loss: 0.682673
Train Epoch: 1 [3000/23974 (12%)]	Loss: 0.568576
Train Epoch: 1 [4000/23974 (17%)]	Loss: 0.827289
Train Epoch: 1 [5000/23974 (21%)]	Loss: 0.507216
Train Epoch: 1 [6000/23974 (25%)]	Loss: 0.541411
Train Epoch: 1 [7000/23974 (29%)]	Loss: 0.702418
Train Epoch: 1 [8000/23974 (33%)]	Loss: 0.641412
Train Epoch: 1 [9000/23974 (38%)]	Loss: 0.717685
Train Epoch: 1 [10000/23974 (42%)]	Loss: 0.622384
Train Epoch: 1 [11000/23974 (46%)]	Loss: 0.684873
Train Epoch: 1 [12000/23974 (50%)]	Loss: 0.523795
Train Epoch: 1 [13000/23974 (54%)]	Loss: 0.577843
Train Epoch: 1 [14000/23974 (58%)]	Loss: 0.649042
Train Epoch: 1 [15000/23974 (62%)]	Loss: 0.733746
Train Epoch: 1 [16000/23974 (67%)]	Loss: 0.646495
Train Epoch: 1 [17000/23974 (71%)]	Loss: 0.694190
Train Epoch: 1 [18000/23974 (75%)]	Loss: 0.658335
Train Epoch: 1 [19000/23974 (79%)]	Loss: 0.756742
Train Epoch: 1 [20000/23974 (83%)]	Loss: 0.520052
Train Epoch

c:\Users\micha\anaconda3\Lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: 0.7751, Accuracy: 6998/10000 (70%)

Round 2/4
Training client 1
Train Epoch: 1 [0/23974 (0%)]	Loss: 1.021651
Train Epoch: 1 [1000/23974 (4%)]	Loss: 0.955496
Train Epoch: 1 [2000/23974 (8%)]	Loss: 0.768518
Train Epoch: 1 [3000/23974 (12%)]	Loss: 0.864623
Train Epoch: 1 [4000/23974 (17%)]	Loss: 0.724258
Train Epoch: 1 [5000/23974 (21%)]	Loss: 0.808818
Train Epoch: 1 [6000/23974 (25%)]	Loss: 0.859125
Train Epoch: 1 [7000/23974 (29%)]	Loss: 0.798802
Train Epoch: 1 [8000/23974 (33%)]	Loss: 0.700376
Train Epoch: 1 [9000/23974 (38%)]	Loss: 0.625857
Train Epoch: 1 [10000/23974 (42%)]	Loss: 0.707647
Train Epoch: 1 [11000/23974 (46%)]	Loss: 0.512070
Train Epoch: 1 [12000/23974 (50%)]	Loss: 0.726743
Train Epoch: 1 [13000/23974 (54%)]	Loss: 0.627127
Train Epoch: 1 [14000/23974 (58%)]	Loss: 0.586230
Train Epoch: 1 [15000/23974 (62%)]	Loss: 0.638155
Train Epoch: 1 [16000/23974 (67%)]	Loss: 0.755404
Train Epoch: 1 [17000/23974 (71%)]	Loss: 0.605010
Train Epoch: 1 [18000/23974 (7

In [41]:
# PCA

rounds_pca = 4

for round_idx in range(rounds_pca):
    
    print(f"Round {round_idx + 1}/{rounds_pca}")

    local_weights_pca = []
    for client_idx, client_model in enumerate(local_models_pca):
        print(f"Training client {client_idx + 1}")
        
        optimizer = optim.SGD(client_model.parameters(), lr=learning_rate,
                      momentum=momentum)

        train_losses = []
        train_counter = []


        for epoch in range(1, n_epochs + 1):  
            train(epoch, client_model, pca_client_loaders_clustered[client_idx], optimizer, log_interval, train_losses, train_counter)
        
        client_weights = [param.data.numpy() for param in client_model.parameters()]
        local_weights_pca.append(client_weights)
        
    print(f"after training{local_models_pca}")
    global_weights_pca = federated_averaging(local_weights_pca)
    print(f"after fedaveraging{local_models_pca}")

    distribute_global_model(global_weights_pca,local_models_pca,single=False)

    distribute_global_model(global_weights_pca,global_model_pca,single=True)
    test_losses = []
    test(global_model_pca,test_loader_pca,test_losses)

Round 1/4
Training client 1
Train Epoch: 1 [0/23974 (0%)]	Loss: 2.485735
Train Epoch: 1 [1000/23974 (4%)]	Loss: 2.201648
Train Epoch: 1 [2000/23974 (8%)]	Loss: 2.083010
Train Epoch: 1 [3000/23974 (12%)]	Loss: 1.977413
Train Epoch: 1 [4000/23974 (17%)]	Loss: 1.920428
Train Epoch: 1 [5000/23974 (21%)]	Loss: 1.600373
Train Epoch: 1 [6000/23974 (25%)]	Loss: 1.571871
Train Epoch: 1 [7000/23974 (29%)]	Loss: 1.504004
Train Epoch: 1 [8000/23974 (33%)]	Loss: 1.142571
Train Epoch: 1 [9000/23974 (38%)]	Loss: 1.258850
Train Epoch: 1 [10000/23974 (42%)]	Loss: 1.290202
Train Epoch: 1 [11000/23974 (46%)]	Loss: 1.225020
Train Epoch: 1 [12000/23974 (50%)]	Loss: 1.164176
Train Epoch: 1 [13000/23974 (54%)]	Loss: 1.057271
Train Epoch: 1 [14000/23974 (58%)]	Loss: 1.046755
Train Epoch: 1 [15000/23974 (62%)]	Loss: 1.053828
Train Epoch: 1 [16000/23974 (67%)]	Loss: 0.865568
Train Epoch: 1 [17000/23974 (71%)]	Loss: 0.989750
Train Epoch: 1 [18000/23974 (75%)]	Loss: 0.820729
Train Epoch: 1 [19000/23974 (79%)]	Los

In [42]:
# Autoencoder

rounds_auto = 4

for round_idx in range(rounds_auto):
    print(f"Round {round_idx + 1}/{rounds_auto}")

    local_weights_auto = []
    for client_idx, client_model in enumerate(local_model_autoencoder):
        print(f"Training client {client_idx + 1}")
        
        optimizer = optim.SGD(client_model.parameters(), lr=learning_rate,
                      momentum=momentum)
        
        train_losses = []
        train_counter = []

        for epoch in range(1, n_epochs + 1):  
            train(epoch, client_model, auto_client_loaders_clustered[client_idx], optimizer, log_interval, train_losses, train_counter)
        
        client_weights = [param.data.numpy() for param in client_model.parameters()]
        local_weights_auto.append(client_weights)
        
    global_weights_auto = federated_averaging(local_weights_auto)

    distribute_global_model(global_weights_auto,local_model_autoencoder,single=False)

    distribute_global_model(global_weights_auto, global_model_auto,single=True)
    test_losses = []
    test(global_model_auto,test_loader_auto,test_losses)

Round 1/4
Training client 1
Train Epoch: 1 [0/23974 (0%)]	Loss: 2.343646
Train Epoch: 1 [1000/23974 (4%)]	Loss: 2.270311
Train Epoch: 1 [2000/23974 (8%)]	Loss: 2.251366
Train Epoch: 1 [3000/23974 (12%)]	Loss: 2.248857
Train Epoch: 1 [4000/23974 (17%)]	Loss: 2.177566
Train Epoch: 1 [5000/23974 (21%)]	Loss: 2.074116
Train Epoch: 1 [6000/23974 (25%)]	Loss: 2.181942
Train Epoch: 1 [7000/23974 (29%)]	Loss: 2.038073
Train Epoch: 1 [8000/23974 (33%)]	Loss: 2.171406
Train Epoch: 1 [9000/23974 (38%)]	Loss: 2.084585
Train Epoch: 1 [10000/23974 (42%)]	Loss: 2.109407
Train Epoch: 1 [11000/23974 (46%)]	Loss: 2.013240
Train Epoch: 1 [12000/23974 (50%)]	Loss: 2.116208
Train Epoch: 1 [13000/23974 (54%)]	Loss: 2.112153
Train Epoch: 1 [14000/23974 (58%)]	Loss: 2.049365
Train Epoch: 1 [15000/23974 (62%)]	Loss: 1.981739
Train Epoch: 1 [16000/23974 (67%)]	Loss: 1.975205
Train Epoch: 1 [17000/23974 (71%)]	Loss: 1.898549
Train Epoch: 1 [18000/23974 (75%)]	Loss: 1.813389
Train Epoch: 1 [19000/23974 (79%)]	Los

# Strong Model Cluster

In [43]:
# Classic

rounds_classic = 4

for round_idx in range(rounds_classic):
    
    print(f"Round {round_idx + 1}/{rounds_classic}")

    local_weights_classic = []
    for client_idx, client_model in enumerate(local_models_classic_strong):
        print(f"Training client {client_idx + 1}")
        
        optimizer = optim.SGD(client_model.parameters(), lr=learning_rate,
                      momentum=momentum)

        train_losses = []
        train_counter = []

        for epoch in range(1, n_epochs + 1):  
            train_fashion(epoch, client_model, classic_client_loaders_clustered[client_idx], optimizer, log_interval, train_losses, train_counter)
        
        client_weights = [param.data.numpy() for param in client_model.parameters()]
        local_weights_classic.append(client_weights)
        

    global_weights_classic = federated_averaging(local_weights_classic)


    distribute_global_model(global_weights_classic,local_models_classic_strong,single=False)

    distribute_global_model(global_weights_classic,global_model_classic_strong,single=True)
    test_losses = []
    test_fashion(global_model_classic_strong,fashion_mnist_test_loader,test_losses)

Round 1/4
Training client 1
Train Epoch: 1 [0/23974 (0%)]	Loss: 0.522776
Train Epoch: 1 [1000/23974 (4%)]	Loss: 0.337348
Train Epoch: 1 [2000/23974 (8%)]	Loss: 0.322143
Train Epoch: 1 [3000/23974 (12%)]	Loss: 0.294087
Train Epoch: 1 [4000/23974 (17%)]	Loss: 0.348672
Train Epoch: 1 [5000/23974 (21%)]	Loss: 0.224583
Train Epoch: 1 [6000/23974 (25%)]	Loss: 0.555698
Train Epoch: 1 [7000/23974 (29%)]	Loss: 0.282498
Train Epoch: 1 [8000/23974 (33%)]	Loss: 0.352552
Train Epoch: 1 [9000/23974 (38%)]	Loss: 0.379134
Train Epoch: 1 [10000/23974 (42%)]	Loss: 0.220214
Train Epoch: 1 [11000/23974 (46%)]	Loss: 0.322660
Train Epoch: 1 [12000/23974 (50%)]	Loss: 0.367432
Train Epoch: 1 [13000/23974 (54%)]	Loss: 0.326016
Train Epoch: 1 [14000/23974 (58%)]	Loss: 0.234246
Train Epoch: 1 [15000/23974 (62%)]	Loss: 0.412466
Train Epoch: 1 [16000/23974 (67%)]	Loss: 0.486033
Train Epoch: 1 [17000/23974 (71%)]	Loss: 0.202997
Train Epoch: 1 [18000/23974 (75%)]	Loss: 0.205094
Train Epoch: 1 [19000/23974 (79%)]	Los

c:\Users\micha\anaconda3\Lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: 0.4239, Accuracy: 8457/10000 (85%)

Round 2/4
Training client 1
Train Epoch: 1 [0/23974 (0%)]	Loss: 0.376190
Train Epoch: 1 [1000/23974 (4%)]	Loss: 0.272269
Train Epoch: 1 [2000/23974 (8%)]	Loss: 0.316284
Train Epoch: 1 [3000/23974 (12%)]	Loss: 0.257268
Train Epoch: 1 [4000/23974 (17%)]	Loss: 0.316200
Train Epoch: 1 [5000/23974 (21%)]	Loss: 0.396427
Train Epoch: 1 [6000/23974 (25%)]	Loss: 0.316117
Train Epoch: 1 [7000/23974 (29%)]	Loss: 0.288723
Train Epoch: 1 [8000/23974 (33%)]	Loss: 0.283512
Train Epoch: 1 [9000/23974 (38%)]	Loss: 0.261743
Train Epoch: 1 [10000/23974 (42%)]	Loss: 0.254122
Train Epoch: 1 [11000/23974 (46%)]	Loss: 0.469816
Train Epoch: 1 [12000/23974 (50%)]	Loss: 0.168087
Train Epoch: 1 [13000/23974 (54%)]	Loss: 0.241160
Train Epoch: 1 [14000/23974 (58%)]	Loss: 0.329779
Train Epoch: 1 [15000/23974 (62%)]	Loss: 0.174850
Train Epoch: 1 [16000/23974 (67%)]	Loss: 0.234565
Train Epoch: 1 [17000/23974 (71%)]	Loss: 0.217776
Train Epoch: 1 [18000/23974 (7

In [44]:
# PCA

rounds_pca = 4

for round_idx in range(rounds_pca):
    
    print(f"Round {round_idx + 1}/{rounds_pca}")

    local_weights_pca = []
    for client_idx, client_model in enumerate(local_models_pca_strong):
        print(f"Training client {client_idx + 1}")
        
        optimizer = optim.SGD(client_model.parameters(), lr=learning_rate,
                      momentum=momentum)

        train_losses = []
        train_counter = []


        for epoch in range(1, n_epochs + 1):  
            train_fashion(epoch, client_model, pca_client_loaders_clustered[client_idx], optimizer, log_interval, train_losses, train_counter)
        
        client_weights = [param.data.numpy() for param in client_model.parameters()]
        local_weights_pca.append(client_weights)
        

    global_weights_pca = federated_averaging(local_weights_pca)

    distribute_global_model(global_weights_pca,local_models_pca_strong,single=False)

    distribute_global_model(global_weights_pca,global_model_pca_strong,single=True)
    test_losses = []
    test_fashion(global_model_pca_strong,test_loader_pca,test_losses)

Round 1/4
Training client 1
Train Epoch: 1 [0/23974 (0%)]	Loss: 0.491333
Train Epoch: 1 [1000/23974 (4%)]	Loss: 0.227463
Train Epoch: 1 [2000/23974 (8%)]	Loss: 0.233770
Train Epoch: 1 [3000/23974 (12%)]	Loss: 0.247685
Train Epoch: 1 [4000/23974 (17%)]	Loss: 0.494265
Train Epoch: 1 [5000/23974 (21%)]	Loss: 0.413155
Train Epoch: 1 [6000/23974 (25%)]	Loss: 0.333106
Train Epoch: 1 [7000/23974 (29%)]	Loss: 0.341256
Train Epoch: 1 [8000/23974 (33%)]	Loss: 0.192805
Train Epoch: 1 [9000/23974 (38%)]	Loss: 0.269251
Train Epoch: 1 [10000/23974 (42%)]	Loss: 0.257719
Train Epoch: 1 [11000/23974 (46%)]	Loss: 0.243606
Train Epoch: 1 [12000/23974 (50%)]	Loss: 0.263083
Train Epoch: 1 [13000/23974 (54%)]	Loss: 0.304439
Train Epoch: 1 [14000/23974 (58%)]	Loss: 0.210762
Train Epoch: 1 [15000/23974 (62%)]	Loss: 0.156424
Train Epoch: 1 [16000/23974 (67%)]	Loss: 0.200947
Train Epoch: 1 [17000/23974 (71%)]	Loss: 0.284574
Train Epoch: 1 [18000/23974 (75%)]	Loss: 0.298043
Train Epoch: 1 [19000/23974 (79%)]	Los

In [45]:
# Autoencoder

rounds_auto = 4

for round_idx in range(rounds_auto):
    print(f"Round {round_idx + 1}/{rounds_auto}")

    local_weights_auto = []
    for client_idx, client_model in enumerate(local_model_autoencoder_strong):
        print(f"Training client {client_idx + 1}")
        
        optimizer = optim.SGD(client_model.parameters(), lr=learning_rate,
                      momentum=momentum)
        
        train_losses = []
        train_counter = []

        for epoch in range(1, n_epochs + 1):  
            train_fashion(epoch, client_model, auto_client_loaders_clustered[client_idx], optimizer, log_interval, train_losses, train_counter)
        
        client_weights = [param.data.numpy() for param in client_model.parameters()]
        local_weights_auto.append(client_weights)
        
    global_weights_auto = federated_averaging(local_weights_auto)

    distribute_global_model(global_weights_auto,local_model_autoencoder_strong,single=False)

    distribute_global_model(global_weights_auto, global_model_auto_strong,single=True)
    test_losses = []
    test_fashion(global_model_auto_strong,test_loader_auto,test_losses)

Round 1/4
Training client 1
Train Epoch: 1 [0/23974 (0%)]	Loss: 0.493551
Train Epoch: 1 [1000/23974 (4%)]	Loss: 0.583331
Train Epoch: 1 [2000/23974 (8%)]	Loss: 0.462129
Train Epoch: 1 [3000/23974 (12%)]	Loss: 0.420153
Train Epoch: 1 [4000/23974 (17%)]	Loss: 0.454641
Train Epoch: 1 [5000/23974 (21%)]	Loss: 0.488483
Train Epoch: 1 [6000/23974 (25%)]	Loss: 0.471783
Train Epoch: 1 [7000/23974 (29%)]	Loss: 0.494689
Train Epoch: 1 [8000/23974 (33%)]	Loss: 0.386161
Train Epoch: 1 [9000/23974 (38%)]	Loss: 0.417625
Train Epoch: 1 [10000/23974 (42%)]	Loss: 0.548303
Train Epoch: 1 [11000/23974 (46%)]	Loss: 0.318957
Train Epoch: 1 [12000/23974 (50%)]	Loss: 0.411810
Train Epoch: 1 [13000/23974 (54%)]	Loss: 0.556645
Train Epoch: 1 [14000/23974 (58%)]	Loss: 0.518286
Train Epoch: 1 [15000/23974 (62%)]	Loss: 0.568789
Train Epoch: 1 [16000/23974 (67%)]	Loss: 0.590449
Train Epoch: 1 [17000/23974 (71%)]	Loss: 0.384189
Train Epoch: 1 [18000/23974 (75%)]	Loss: 0.298780
Train Epoch: 1 [19000/23974 (79%)]	Los